In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# 데이터 로드
data = pd.read_csv("./merged_data.csv")

# weekday 컬럼에 대한 one-hot 인코딩
data_encoded = pd.get_dummies(data, columns=['weekday'])

# 입력 변수와 타겟 변수 분리
X = data_encoded[['time', '1시간기온', '1시간강수량', '일최고기온', '일최저기온', 
                  'weekday_Friday', 'weekday_Monday', 'weekday_Saturday', 
                  'weekday_Sunday', 'weekday_Thursday', 'weekday_Tuesday', 'weekday_Wednesday']]
y = data_encoded.loc[:, 'SV-2_석사':'산학협력연구동(E)_학사']

# 학습 세트와 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# 랜덤 포레스트 회귀 모델 초기화 및 학습
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
rf_regressor.fit(X_train, y_train)

# 테스트 세트에서의 예측 수행
y_pred = rf_regressor.predict(X_test)

# 평균 제곱 오차 계산
mse = mean_squared_error(y_test, y_pred, multioutput='raw_values')


In [ ]:
# 테스트 데이터 예측
y_pred_test = rf_regressor.predict(X_test)

# 예측값을 DataFrame으로 변환
df_predictions_test = pd.DataFrame(y_pred_test, columns=y_test.columns, index=y_test.index)

# 실제값과 예측값을 나란히 출력
comparison_list_test = []
for column in y_test.columns:
    comparison_list_test.append(y_test[column])
    comparison_list_test.append(df_predictions_test[column])

formatted_comparison_df_test = pd.concat(comparison_list_test, axis=1)
print(formatted_comparison_df_test)

In [6]:
# 테스트 세트에 대한 예측값의 합계 계산
total_load = y_pred_test.sum(axis=1)

# 'day', 'time', 'load' 형태의 데이터 생성
load_df = X_test[['time']].copy()
load_df['day'] = data_encoded.loc[X_test.index, 'day']
load_df['load'] = total_load

# 'day', 'time', 'load' 순서로 컬럼 재배열
load_df = load_df[['day', 'time', 'load']]

load_df.head()


,day,time,load
1318,2022-08-24,22,28895.246
899,2022-08-07,11,30012.126
439,2022-07-19,7,28144.893
560,2022-07-24,8,28244.253
433,2022-07-19,1,28067.142
